## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://geemap.org). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/giswqs/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import ee
import geemap
import pprint

# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

## Load Landsat Collection for Specified Region

In [3]:
import os
os.environ['PROJ_LIB'] = r'/mnt/lustre/a2fs-work3/work/e283/e283/leitm/yhe/venv/miniconda3/envs/jupyter-gee/share/proj'

import sys
sys.path.insert(0, "./lst_cam_modules")

import utils

# geometry = utils.getGeometry('users/yhephd/coords/coord_delhi_ronita')

geo_shp = './data/delhi/shp/delhi_boundary.shp'
# geo_shp = './data/delhi/shp/delhi_districts.shp'
geometry = geemap.shp_to_ee(geo_shp).geometry()

# mencari image
delhiColl = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
         .filterBounds(geometry) \
         .filterDate('2022-04-20','2022-04-30') \
         .filterMetadata('CLOUD_COVER', 'less_than', 25)

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

delhiColl = delhiColl.map(applyScaleFactors)

pp = pprint.PrettyPrinter(depth=3)
pp.pprint(delhiColl.getInfo())

{'bands': [],
 'features': [{'bands': [...],
               'id': 'LANDSAT/LC08/C02/T1_L2/LC08_146040_20220420',
               'properties': {...},
               'type': 'Image',
               'version': 1651191286933086},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C02/T1_L2/LC08_146041_20220420',
               'properties': {...},
               'type': 'Image',
               'version': 1651193655085863},
              {'bands': [...],
               'id': 'LANDSAT/LC08/C02/T1_L2/LC08_147040_20220427',
               'properties': {...},
               'type': 'Image',
               'version': 1651688584717690.0}],
 'id': 'LANDSAT/LC08/C02/T1_L2',
 'properties': {'date_range': [1365638400000, 1654560000000],
                'description': '<p>This dataset contains atmospherically '
                               'corrected\n'
                               'surface reflectance and land surface '
                               'temperature derived from the 

## Select a Landsat Image for processing

In [4]:
listOfImages = delhiColl.toList(delhiColl.size())
image = ee.Image(listOfImages.get(0))
pp = pprint.PrettyPrinter(depth=3)
pp.pprint(image.getInfo())

{'bands': [{'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B1'},
           {'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B2'},
           {'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B3'},
           {'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B4'},
           {'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B5'},
           {'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'SR_B6'},
           {'crs': 'EPSG:326

## Calculate & Display NDVI

In [5]:
Map = geemap.Map()
Map.centerObject(geometry, 10)

vizParams1 = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'Max': 0.3,
}
Map.addLayer(image, vizParams1, 'RGB', 0)

ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
pp = pprint.PrettyPrinter(depth=3)
pp.pprint(ndvi.getInfo())

pp = pprint.PrettyPrinter(depth=1)
minNDVI = ee.Number(ndvi.select('NDVI').reduceRegion(
    reducer = ee.Reducer.min(), 
    geometry = geometry, 
    scale = 30, 
    maxPixels = 1e9 
).values().get(0))
pp.pprint('minNDVI: '+str(minNDVI.getInfo()))
maxNDVI = ee.Number(ndvi.select('NDVI').reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = geometry,
    scale = 30,
    maxPixels = 1e9
).values().get(0))
pp.pprint('maxNDVI: '+str(maxNDVI.getInfo()))

visParams2 = {
    'min': minNDVI.getInfo(),
    'max': maxNDVI.getInfo(),
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red'],
}
Map.addLayer(ndvi, visParams2, 'NDVI')
Map.add_colorbar(visParams2, label='NDVI')

Map.centerObject(geometry, 10)
Map.addLayer(geometry, {}, 'geometry', 1, 0.5)
Map

{'bands': [{'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'NDVI'}],
 'properties': {'system:footprint': {'coordinates': [...],
                                     'type': 'LinearRing'}},
 'type': 'Image'}
'minNDVI: -0.11264036262490987'
'maxNDVI: 0.8125402834136732'


Map(center=[28.641157799909436, 77.11893372419047], controls=(WidgetControl(options=['position', 'transparent_…

## Calculate & Display LST

In [6]:
Map = geemap.Map()
Map.centerObject(geometry, 10)

# Compute FVC
fvc = image.expression('((ndvi-vmin)/(vmax-vmin))**2',
  {'ndvi':ndvi,'vmin':minNDVI,'vmax':maxNDVI}).rename('FVC')
fvc = fvc.where(fvc.lt(0.0),0.0)
fvc = fvc.where(fvc.gt(1.0),1.0)
Map.addLayer(fvc, {}, 'FVC', 0)

# Emissivity
EM = image.expression('a*fvc+b',
  {'fvc':fvc,'a':0.004,'b':0.986}).rename('EMM')
Map.addLayer(EM, {},'EMM', 0)

LST = image.expression('(Tb/(1 + (0.00115* (Tb / 1.438))*log(Ep)))-273.15', 
 {'Tb': image.select('ST_B10'), 'Ep': EM.select('EMM')}).rename('LST');

pp = pprint.PrettyPrinter(depth=3)
pp.pprint(LST.getInfo())

pp = pprint.PrettyPrinter(depth=1)
minLST = ee.Number(LST.select('LST').reduceRegion(
    reducer = ee.Reducer.min(), 
    geometry = geometry, 
    scale = 30, 
    maxPixels = 1e9 
).values().get(0))
pp.pprint('minLST: '+str(minLST.getInfo()))
maxLST = ee.Number(LST.select('LST').reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = geometry,
    scale = 30,
    maxPixels = 1e9
).values().get(0))
pp.pprint('maxLST: '+str(maxLST.getInfo()))

visParams3 = {
    'min': minLST.getInfo(),
    'max': maxLST.getInfo()-15,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red'],
}
Map.addLayer(LST, visParams3,'LST', 1)
Map.add_colorbar(visParams3, label='LST (C)')

Map.addLayer(geometry, {}, 'geometry', 1, 0.5)
Map

{'bands': [{'crs': 'EPSG:32643',
            'crs_transform': [...],
            'data_type': {...},
            'dimensions': [...],
            'id': 'LST'}],
 'type': 'Image'}
'minLST: 30.00336983455611'
'maxLST: 77.14880003220856'


Map(center=[28.641157799909436, 77.11893372419047], controls=(WidgetControl(options=['position', 'transparent_…

## Export LST Image

In [7]:
# Export the image, specifying scale and region.
# See GEE to check upload status
task = ee.batch.Export.image.toDrive(**{
  'image': LST.select(['LST']),
  'description': 'Delhi-LC08_146040_20220420-LST_30m',
  'folder': 'archer2gee',
  'scale': 30,
  'region': geometry,
  'fileFormat': 'GeoTIFF',
  'crs': 'EPSG:5070',
  'maxPixels': 1e13
})
task.start()
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
Polling for task (id: YC572GWKFOHAMVXFV7X5RV72).
